In [7]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records, JoinType 
from credmark.cmf.types.data.erc_standard_data import ERC20_BASE_ABI
from models.tmp_abi_lookup import RAIN_LOC, ESCROW 
import datetime


cmf_param = {
    'chain_id': 1,
}

context, _model_loader = create_cmf(cmf_param)

In [18]:
RainCCLoC = Contract('0xbf2d49ecfe657132f34863263d654d8e2eb1d72e').set_abi(RAIN_LOC, set_loaded=True)

escrow = RainCCLoC.functions.escrow().call()

escrowContract = Contract(escrow).set_abi(ESCROW, set_loaded=True)

collateral = escrowContract.functions.getCollateralValue().call()

id = RainCCLoC.functions.ids(0).call()

credit = RainCCLoC.functions.credits(id).call()


# interstAccrued = RainCCLoC.functions.interestAccrued(id).call()

# avaialbleCredit = RainCCLoC.functions.available(id).call()

# deadline = RainCCLoC.functions.deadline().call()

# arbiter = RainCCLoC.functions.arbiter().call()


[interstAccrued, avaialbleCredit, deadline, normalDeadline] = context.web3_batch.call([
    RainCCLoC.functions.interestAccrued(id),
    RainCCLoC.functions.available(id),
    RainCCLoC.functions.deadline(),
    RainCCLoC.functions.arbiter()
], unwrap=True)

normalDeadline = datetime.datetime.fromtimestamp(deadline, datetime.UTC)


token = Token(credit[5]).set_abi(ERC20_BASE_ABI, set_loaded=True)

# token_price = context.run_model(
#                     slug='price.quote',
#                     input={'base': token})['price']


# 0 - deposit, 1 - principal, 2 - interestAccrued, 3 - interestRepaid, 4 - decimals, 5 - token, 6 - lender, 7 - isOpen

print(f"Token : {token.symbol} {token.name} {token.decimals}")
# print(f"Token price: {token_price}")
print("Credit Token: ", credit[5])
print("Outstanding principal (USDC):", credit[1]/(10**credit[4]))
print("Accrued Interest (USDC):", interstAccrued/(10**credit[4]))
print("Total Outstanding Debt (USDC):", (credit[1] + interstAccrued)/(10**credit[4]))
print("Funds Available to Draw (USDC):", avaialbleCredit[0]/(10**credit[4]))
print("Interest Available to Withdraw (USDC):", avaialbleCredit[1]/(10**credit[4]))
print("Dealdine: ", normalDeadline)
print("Collateral: ", collateral)


Token : USDC USD Coin 6
Credit Token:  0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
Outstanding principal (USDC): 600000.0
Accrued Interest (USDC): 1669.88364
Total Outstanding Debt (USDC): 601669.88364
Funds Available to Draw (USDC): 0.0
Interest Available to Withdraw (USDC): 6140.349075
Dealdine:  2024-04-12 20:22:23+00:00
Collateral:  0
